# Exercise 2: Transform the recorded robot positions into a Self Organizing Map (SOM)

The goal of this exercise is to generate a Self Organizing Map based on the recorded robot positions of Exercise 1.

If you didn't complete Exercise 1, you can clone and launch the experiment *"Exercise 2: Transform robot positions ... (SOM)"* of the [Neurorobotics platform](http://148.187.97.48/#/esv-private) to generate the file *robot_positions.csv* and copy it this Collab's Storage.

**Note that Exercise 2's experiment contains the solution of Exercise 1.**

In [ ]:
# Download the file robot_positions.csv file from this Collab's Storage
# If you didn't upload the file obtained in Exercise 1, a default robot_positions.csv file is provided
! pip install --upgrade "hbp-service-client==1.1.1"
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
clients.storage.download_file(collab_path + '/robot_positions.csv', './robot_positions.csv')

## A. Complete the SOM implementation
---------------------------------------------------

In this section, you will complete the inplementation of the SOM algorithm.

Your script will be saved to the file `SOM_<Surname>_<Name>.py` for its submition via the command: 
    %%writefile SOM_Surname_Name.py
    
Please, replace `<Surname>` and `<Name>` in the name of the file to ensure autorecognition of your personality
Once you are ready to submit your SOM implementation, you need to uncomment the first line so as to create your file in the jupyter notebook environment.

Remarks:
- 1) The map size shouldn't exceed 10x10. It's not reasonable to use very huge map for our simple environment.
- 2) The theory says the 1 minute of simulation is totally enough for SOM training in our experiment.

In [ ]:
# Install SOM utils
! pip install --user --force-reinstall git+https://github.com/HBPNeurorobotics/EPFLx-RoboX-Neurorobotics-utils#egg=epflx_robox_nrp_utils

In [ ]:
#%%writefile SOM_Surname_Name.py 
# class SOM (SOM_solution)

import numpy as np
import random

from IPython import display



class SOM():

	# Self-Organizing Map mapping the environment depending on the positions visited by the robot
    
	def __init__(self, video=0, csv_file='robot_positions.csv'):
		# Inputs:
		#	 Nn: size of the 2D lattice (Nn*Nn)
		#	 eta: learning rate
		#	 sigma_0: initial width of the neighborhood function
		#	 tau_sigma: mean life time of width (decaying exponential)
		#	 sigma_min: width minimum value


		''' TO DO: set parameters of SOM training '''
		self.Nn = 12
		self.eta_0 = 1.0
		self.sigma_0 = 3.0
        
		self.eta_min = 0.001
		self.sigma_min = 0.0
        
		self.tau = 100.0
		''' --------------- TO DO --------------- '''
    

		self.pos = {}
		self.mix = {}
		self.trial = 0 # number of updates (eta and sigma can depend on the number of updates)
		self.video = video
		self.csv_file = csv_file
        
		# Lattice of neurons (SOM) of size [Nn,Nn]. The third dimension allows to store x and y coordinates.
		# The values are randomly initialized
		self.lattice = np.random.uniform(0.,1.0,(self.Nn,self.Nn,2))
        

    
    
	### Run simulation (main function) ###
        
	def run_som(self):
		'''----- Additional functions: upload, visualize, save ----- '''
		from epflx_robox_nrp_utils.SOM.SOM_additional import SOM_additional
		somad = SOM_additional() 
		'''--------------------------------------------------------- '''
        
		self.pos = somad.load_data(self.csv_file)
		while(self.trial < self.tau):
			self.run_trial()
			somad.save_lattice(self.lattice)
			if(self.video): somad.visualization(self.lattice,self.Nn,self.eta(),self.sigma(),self.trial)
			self.trial += 1
		display.clear_output(wait=True)



	### SOM training stages ###

	def run_trial(self):
		self.mix = self.datamix();
		for i in range(self.pos.shape[0]):
			self.run_episode(i)


	def run_episode(self,i):
		pt = self.pos[self.mix[i]]
		d = self.distance(pt)
		[lx,ly] = self.winner(d)
		h = self.neighborhood_factor(lx[0],ly[0])
		self.update_lattice(h,pt)


	def update_lattice(self,h,pt):
		for i in range(self.lattice.shape[2]):
			self.lattice[:,:,i] += self.eta()*h*((pt-self.lattice)[:,:,i])



	### Neighborhood factor ###
        
	def distance(self, pt):
		return np.linalg.norm(self.lattice - pt,axis=2)**2
    
    
	def winner(self,d):
		return np.where(d == d.min())
        
    
	def neighborhood_factor(self,lx,ly):
		h = np.zeros((self.Nn,self.Nn))
		for i in range(self.Nn):
			for j in range(self.Nn):
				dist = (i-lx)**2 + (j-ly)**2
				h[i,j] = np.exp(-dist/(2*self.sigma()**2))
		return h



	### Parameters ###
        
	def eta(self):
		return self.eta_0*(1-self.trial/self.tau)
    
    
	def sigma(self):
		return self.sigma_0*(1-self.trial/self.tau)
        
        
	def datamix(self):
		mix = []
		for j in range(self.pos.shape[0]):
			mix.append(random.randint(0,self.pos.shape[0]-1))
		return mix

## B. Perform SOM training
----------------------------------

In [ ]:
# from SOM_Surname_Name import SOM
##som = SOM(visualization, data file [The provided file robot_positions.csv is used by default])
import time
T = time.time()
som = SOM(0)
som.run_som()
print time.time() - T

## C. Evaluation of result (for non-graduated students) 
---------------------------------------------------------------------

In [ ]:
# Variation: Estimate result of SOM training
from epflx_robox_nrp_utils.SOM.SOM_evaluation import SOM_evaluation
somev = SOM_evaluation()
var, N = somev.run_evaluation()
print "Your result is equal to", var, "with SOM (", N, "x", N,")."

## =============================  SOM evaluation  ============================

## SOM evaluation (for only one graduated student) 

In [ ]:
from SOM_autograduation import SOM_autograduation
somag = SOM_autograduation()
somag.one_function_graduation('SOM_Marc-Oliver_Gewaltig.py')

## SOM evaluation (for all graduated students) 

In [ ]:
from SOM_autograduation import SOM_autograduation
somag = SOM_autograduation()
somag.all_functions_graduation()

## Display graduation results 

In [ ]:
from SOM_autograduation import SOM_autograduation
somag = SOM_autograduation()
somag.open_webpage()